In [1]:
from mpa.models import Mpa, mpa_post_save
from django.db.models.signals import post_save, post_delete
from mpa.views import mpas_all_nogeom
import reversion
from reversion.models import Revision
from reversion.models import Version
import datetime
from django.db.models import Q, F, Func
from wdpa.models import Wdpa2019Poly, Wdpa2019Point

In [2]:
mpaset = mpas_all_nogeom.exclude(
        Q(country__icontains='USA') | Q(sovereign__icontains='USA') |
        Q(country__icontains='UMI') | Q(sovereign__icontains='UMI') |
        Q(country__icontains='VIR') | Q(sovereign__icontains='VIR') |
        Q(country__icontains='PRI') | Q(sovereign__icontains='PRI') |
        Q(country__icontains='ASM') | Q(sovereign__icontains='ASM') |
        Q(country__icontains='GUM') | Q(sovereign__icontains='GUM') |
        Q(country__icontains='MNP') | Q(sovereign__icontains='MNP')
    ).exclude(
        Q(country__icontains='MEX') | Q(country__icontains='CHL') | Q(country__icontains='IDN') | 
        Q(country__icontains='CHN') | Q(country__icontains='JPN') | 
        Q(sovereign__icontains='MEX') | Q(sovereign__icontains='CHL') | Q(sovereign__icontains='IDN') |
        Q(sovereign__icontains='CHN') | Q(sovereign__icontains='JPN')
    ).filter(Q(country__icontains='TUV') | Q(sovereign__icontains='TUV'))
mpaset.values('country').distinct()

<QuerySet [{'country': 'TUV'}]>

In [4]:
# mpaset = mpas_all_nogeom.exclude(
#         Q(country__icontains='USA') | Q(sovereign__icontains='USA') |
#         Q(country__icontains='UMI') | Q(sovereign__icontains='UMI') |
#         Q(country__icontains='VIR') | Q(sovereign__icontains='VIR') |
#         Q(country__icontains='PRI') | Q(sovereign__icontains='PRI') |
#         Q(country__icontains='ASM') | Q(sovereign__icontains='ASM') |
#         Q(country__icontains='GUM') | Q(sovereign__icontains='GUM') |
#         Q(country__icontains='MNP') | Q(sovereign__icontains='MNP')
#     ).exclude(
#         Q(country__icontains='MEX') | Q(country__icontains='CHL') | Q(country__icontains='IDN') | 
#         Q(country__icontains='CHN') | Q(country__icontains='JPN') | 
#         Q(sovereign__icontains='MEX') | Q(sovereign__icontains='CHL') | Q(sovereign__icontains='IDN') |
#         Q(sovereign__icontains='CHN') | Q(sovereign__icontains='JPN')
#     ).exclude(
#         Q(country__icontains='FRA') | Q(sovereign__icontains='FRA') |
#         Q(country__icontains='ATF') | Q(country__icontains='BLM') | Q(country__icontains='GLP') |
#         Q(country__icontains='GUF') | Q(country__icontains='MAF') | Q(country__icontains='MTQ') |
#         Q(country__icontains='MYT') | Q(country__icontains='NCL') | Q(country__icontains='PYF') |
#         Q(country__icontains='REU') | Q(country__icontains='SHN') | Q(country__icontains='SYC')
#     ).exclude(
#         Q(country__icontains='AUS') | Q(sovereign__icontains='AUS') |
#         Q(country__icontains='ZAF') | Q(sovereign__icontains='ZAF') |
#         Q(country__icontains='CAN') | Q(sovereign__icontains='CAN')
#     ).exclude(
#         Q(country__icontains='GBR') | Q(sovereign__icontains='GBR') |
#         Q(country__icontains='AIA') | Q(country__icontains='BMU') | Q(country__icontains='CYM') |
#         Q(country__icontains='FLK') | Q(country__icontains='GIB') | Q(country__icontains='IMN') |
#         Q(country__icontains='IOT') | Q(country__icontains='JEY') | Q(country__icontains='MSR') |
#         Q(country__icontains='PCN') | Q(country__icontains='SGS') | Q(country__icontains='SHN') |
#         Q(country__icontains='TCA') | Q(country__icontains='VGB')
#     )
# mpaset.values('country').distinct()

In [3]:
mpas_all_nogeom.filter(wdpa_id=0).update(wdpa_id=None)
mpas_all_nogeom.filter(wdpa_id=0).count()

0

In [4]:
count = 0
# cutoff_date = datetime.datetime(2014, 11, 5)
cutoff_date = datetime.datetime(2018, 10, 15)
for m in mpaset:
    versions = Version.objects.get_for_object(m)
    created = 'Never'
    if not m.wdpa_id: continue
    try:
        created = versions.first().revision.date_created
        if created > cutoff_date:
            count = count + 1
            print(count, created, 'count:', len(versions), 'mpa_id:', m.mpa_id, 'wdpa_id:', m.wdpa_id)
    except:
        pass

1 2019-12-31 14:26:21.608063 count: 2 mpa_id: 68815949 wdpa_id: 555547861
2 2019-12-31 14:54:02.714215 count: 2 mpa_id: 68817107 wdpa_id: 555651495
3 2019-12-31 14:26:24.508178 count: 2 mpa_id: 68815951 wdpa_id: 555547862
4 2019-12-31 14:26:31.883866 count: 2 mpa_id: 68815955 wdpa_id: 555547865
5 2019-12-31 14:26:19.513372 count: 4 mpa_id: 14508 wdpa_id: 555547861
6 2020-01-01 13:09:46.809052 count: 6 mpa_id: 6846 wdpa_id: 145496
7 2019-12-31 14:53:59.748670 count: 2 mpa_id: 68817104 wdpa_id: 555651493
8 2019-12-31 14:54:00.840096 count: 2 mpa_id: 68817105 wdpa_id: 555651493
9 2019-12-31 14:26:26.758479 count: 2 mpa_id: 68815952 wdpa_id: 555547863
10 2019-12-31 14:26:28.708839 count: 2 mpa_id: 68815953 wdpa_id: 555547864
11 2019-12-31 14:26:29.763878 count: 2 mpa_id: 68815954 wdpa_id: 555547864
12 2019-12-31 14:26:23.653076 count: 2 mpa_id: 68815950 wdpa_id: 555547862
13 2019-12-31 14:26:25.894306 count: 4 mpa_id: 14510 wdpa_id: 555547863
14 2019-12-31 14:26:27.774412 count: 4 mpa_id: 

In [5]:
from importlib import reload
from wdpa import merge
reload(merge)

<module 'wdpa.merge' from '/home/mpatlas/projects/django_mpatlas/wdpa/merge.py'>

In [47]:
tas = Wdpa2019Poly.objects.filter(wdpa_pid__icontains='345888')
print(tas.count())
for t in tas:
    print(t.wdpaid, t.wdpa_pid, t.name, ',', t.desig, t.marine)

sgs = Wdpa2019Poly.objects.filter(name__icontains='South Georgia')
print(sgs.count())
for s in sgs:
    print(s.wdpaid, s.wdpa_pid, s.name, ',', s.desig, s.marine)


1
345888.0 345888 Terres Australes Françaises , Réserve Naturelle Nationale 2
1
555547601.0 555547601 South Georgia and South Sandwich Islands Marine Protected Area , Marine Protected Area 2


In [6]:
# remove_log = '/home/mpatlas/workspace/log/removewdpa_TUV_log.json'
remove_log = '/Users/russmo/Code/wdpa/log/removewdpa_TUV_2019_log.json'
wdpa2remove = merge.getRemoveWdpaList(logfile=remove_log)

In [7]:
len(wdpa2remove)

0

In [8]:
rm_mpas = Mpa.objects.filter(wdpa_id__in=wdpa2remove).order_by('country')
len(rm_mpas)

0

In [9]:
for m in rm_mpas:
    print(m.mpa_id, m.wdpa_id, m.wdpa_pid, m.country, m.sovereign, m.is_mpa, m.no_take, m.no_take_area, m.name, m.designation_eng)

In [10]:
merge.removeMpasByWdpaId(wdpa2remove)

In [11]:
reload(merge)

<module 'wdpa.merge' from '/home/mpatlas/projects/django_mpatlas/wdpa/merge.py'>

In [12]:
wdpa2add = merge.getAddWdpaList()

In [13]:
len(wdpa2add)

9

In [14]:
wdpapid2add = merge.getAddWdpaPidList(verbose=True)

15 wdpa_pid's processed
555547861_C : 1 processed
555547862_A : 2 processed
555547865_B : 3 processed
555547862_B : 4 processed
555547861_A : 5 processed
555547862_C : 6 processed
555547864_B : 7 processed
555547864_A : 8 processed
555547864_C : 9 processed
555651493_B : 10 processed
555651493_A : 11 processed
555547865_A : 12 processed
555547861_B : 13 processed
555547863_B : 14 processed
555547863_A : 15 processed


In [15]:
len(wdpapid2add)

15

In [18]:
# Assign records with zones to be the first zone.  Do not rerun.
# In separate notebook

In [16]:
allpid2add = wdpapid2add + [str(int(i)) for i in wdpa2add]
len(allpid2add)
Wdpa2019Poly.objects.filter(wdpa_pid__in=allpid2add).count() + \
Wdpa2019Point.objects.filter(wdpa_pid__in=allpid2add).count()

18

In [17]:
unmatched_pids_poly = Wdpa2019Poly.objects.filter(
    Q(wdpa_pid__in=allpid2add) | Q(wdpaid__in=wdpa2add)
).exclude(
    wdpa_pid=Func(F('wdpaid'), function='INTEGER', template='(%(expressions)s::%(function)s)::text')
)
unmatched_pids_point = Wdpa2019Point.objects.filter(
    Q(wdpa_pid__in=allpid2add) | Q(wdpaid__in=wdpa2add)
).exclude(
    wdpa_pid=Func(F('wdpaid'), function='INTEGER', template='(%(expressions)s::%(function)s)::text')
)
unmatched_pids_poly.order_by('wdpa_pid').values_list('wdpaid', 'wdpa_pid')


<QuerySet [(555547861.0, '555547861_A'), (555547861.0, '555547861_B'), (555547861.0, '555547861_C'), (555547862.0, '555547862_A'), (555547862.0, '555547862_B'), (555547862.0, '555547862_C'), (555547863.0, '555547863_A'), (555547863.0, '555547863_B'), (555547864.0, '555547864_A'), (555547864.0, '555547864_B'), (555547864.0, '555547864_C'), (555547865.0, '555547865_A'), (555547865.0, '555547865_B'), (555651493.0, '555651493_A'), (555651493.0, '555651493_B')]>

In [18]:
# Set wdpa_pid for new zones
# If existing MPAtlas sites with this wdpa_id,
# set first non-rejected site to the first PID_A.
# Clear pid for all other MPAtlas sites and flag
# as orphaned zones.

wzones = []
mpasw = [float(wi) for wi in mpaset.filter(wdpa_id__isnull=False).order_by('wdpa_id').values_list('wdpa_id', flat=True)]
polyset = Wdpa2019Poly.objects.filter(wdpaid__in=mpasw).only('wdpaid', 'wdpa_pid', 'name', 'desig_eng', 'iso3', 'parent_iso3')
for w in polyset:
    if w.wdpa_pid != str(int(w.wdpaid)):
        wzones.append(w.wdpaid)
wzones = list(set(wzones))
len(wzones)

6

In [19]:
# Assign mpa records that were previously unzoned to have the pid of the first zone in the updated WDPA data set for this site.
count = 0
for wid in wzones:
    zones = Wdpa2019Poly.objects.filter(wdpaid=wid).only('wdpaid', 'wdpa_pid', 'name', 'desig_eng', 'iso3', 'parent_iso3')
    firstpid = zones.order_by('wdpa_pid').first().wdpa_pid
    mpaz = mpaset.filter(wdpa_id=int(wid)).order_by('mpa_id')
    m = None
    if mpaz.filter(wdpa_pid=firstpid).count() == 0:
        # no mpas match first zone pid
        if mpaz.count() == 1:
            if mpaz[0].wdpa_pid == str(mpaz[0].wdpa_id) or mpaz[0].wdpa_pid == '' or mpaz[0].wdpa_pid is None:
                # only record without specific zone, so go ahead and set it first zone
                m = mpaz[0]
        else:
            if mpaz.count() == mpaz.filter(Q(wdpa_pid=Func(F('wdpa_id'), function='INTEGER', template='(%(expressions)s::%(function)s)::text')) | Q(wdpa_pid__isnull=True) | Q(wdpa_pid='')).count():
                # more than one mpatlas mpa record with this wdpa_id
                # but none have a proper wdpa_pid different from the wdpa_id
                # so set first one to have the first zone pid
                nonrejects = mpaz.exclude(verification_state='Rejected as MPA').order_by('mpa_id')
                if nonrejects:
                    m = nonrejects[0]
                else:
                    m = mpaz[0]
    if m:
        print(m.mpa_id, m.wdpa_id, m.wdpa_pid)
        m.wdpa_pid = firstpid
        count += 1
        print(count, m.mpa_id, m.wdpa_id, m.wdpa_pid)
        m.save()


In [20]:
reload(merge)
addpoint_geolog = '/home/mpatlas/workspace/log/addpoint_TUV_2019_log.json'
addpoly_log = '/home/mpatlas/workspace/log/addpoly_TUV_2019_log.json'
addpoly_geolog = '/home/mpatlas/workspace/log/addpoly_TUV_2019_geo_log.json'
# addpoint_geolog = '/Users/russmo/Code/wdpa/log/addpoint_TUV_2019_log.json'
# addpoly_log = '/Users/russmo/Code/wdpa/log/addpoly_TUV_2019_v2_zerogeofix_log.json'
# addpoly_geolog = '/Users/russmo/Code/wdpa/log/addpoly_TUV_2019_v2_zerogeofix_geo_log.json'

In [21]:
points = Wdpa2019Point.objects.filter(wdpa_pid__in=allpid2add).order_by('wdpa_pid')
merge.updateMpasFromWdpaQueryset(qs=points, poly=False, logfile=None, geologfile=addpoint_geolog, dryrun=False)

True

In [22]:
# post_save.disconnect(mpa_post_save, sender=Mpa)
polys = Wdpa2019Poly.objects.filter(wdpa_pid__in=allpid2add).order_by('wdpa_pid')
merge.updateMpasFromWdpaQueryset(qs=polys, poly=True, logfile=addpoly_log, geologfile=None, dryrun=False)
# post_save.connect(mpa_post_save, sender=Mpa)

1/18 adding/updating wdpa_pid 145496 with mpa_id 6846
    INFO: Older WDPA record not found
    MPAtlas orig geom area is zero, using WDPA boundaries
    Retained old MPAtlas values for: 
2/18 adding/updating wdpa_pid 555547861_A with mpa_id 14508
    INFO: Older WDPA record not found
    MPAtlas orig geom area is zero, using WDPA boundaries
    Retained old MPAtlas values for: 
3/18 adding/updating wdpa_pid 555547861_B with mpa_id 68815948
    INFO: Older WDPA record not found
    MPAtlas orig geom area is zero, using WDPA boundaries
    Retained old MPAtlas values for: 
4/18 adding/updating wdpa_pid 555547861_C with mpa_id 68815949
    INFO: Older WDPA record not found
    MPAtlas orig geom area is zero, using WDPA boundaries
    Retained old MPAtlas values for: 
5/18 adding/updating wdpa_pid 555547862_A with mpa_id 14509
    INFO: Older WDPA record not found
    MPAtlas orig geom area is zero, using WDPA boundaries
    Retained old MPAtlas values for: 
6/18 adding/updating wdpa_pid 

True

In [21]:
bad = Wdpa2019Poly.objects.get(wdpa_pid='555643543')
bg = bad.geom
bg.num_coords
555643544
bad = Wdpa2019Poly.objects.get(wdpa_pid='555643544')
bg = bad.geom
bg.num_coords

2833100

In [23]:
reload(merge)
updatepoint_geolog = '/home/mpatlas/workspace/log/updatepoint_TUV_log.json'
updatepoly_log = '/home/mpatlas/workspace/log/updatepoly_TUV_log.json'
updatepoly_geolog = '/home/mpatlas/workspace/log/updatepoly_TUV_geo_log.json'
# updatepoint_geolog = '/Users/russmo/Code/wdpa/log/updatepoint_TUV_2019_v3_geo_log.json'
# updatepoly_log = '/Users/russmo/Code/wdpa/log/updatepoly_TUV_2019_v3_log.json'
# updatepoly_geolog = '/Users/russmo/Code/wdpa/log/updatepoly_TUV_2019_v3_geo_log.json'

In [26]:
# wdpa_filter = (
# #     Q(iso3__icontains='USA') | Q(parent_iso3__icontains='USA') |
# #     Q(iso3__icontains='UMI') | Q(parent_iso3__icontains='UMI') |
# #     Q(iso3__icontains='VIR') | Q(parent_iso3__icontains='VIR') |
# #     Q(iso3__icontains='PRI') | Q(parent_iso3__icontains='PRI') |
# #     Q(iso3__icontains='ASM') | Q(parent_iso3__icontains='ASM') |
# #     Q(iso3__icontains='GUM') | Q(parent_iso3__icontains='GUM') |
# #     Q(iso3__icontains='MNP') | Q(parent_iso3__icontains='MNP') |
#     Q(iso3__icontains='MEX') | Q(iso3__icontains='CHL') | Q(iso3__icontains='IDN') | 
#     Q(iso3__icontains='CHN') | Q(iso3__icontains='JPN') | 
#     Q(parent_iso3__icontains='MEX') | Q(parent_iso3__icontains='CHL') | Q(parent_iso3__icontains='IDN') |
#     Q(parent_iso3__icontains='CHN') | Q(parent_iso3__icontains='JPN') #|
# #     Q(iso3__icontains='FRA') | Q(parent_iso3__icontains='FRA') |
# #     Q(iso3__icontains='ATF') | Q(iso3__icontains='BLM') | Q(iso3__icontains='GLP') |
# #     Q(iso3__icontains='GUF') | Q(iso3__icontains='MAF') | Q(iso3__icontains='MTQ') |
# #     Q(iso3__icontains='MYT') | Q(iso3__icontains='NCL') | Q(iso3__icontains='PYF') |
# #     Q(iso3__icontains='REU') | Q(iso3__icontains='SHN') | Q(iso3__icontains='SYC') |
# #     Q(iso3__icontains='AUS') | Q(parent_iso3__icontains='AUS') |
# #     Q(iso3__icontains='ZAF') | Q(parent_iso3__icontains='ZAF') |
# #     Q(iso3__icontains='CAN') | Q(parent_iso3__icontains='CAN') |
# #     Q(iso3__icontains='GBR') | Q(parent_iso3__icontains='GBR') |
# #     Q(iso3__icontains='AIA') | Q(iso3__icontains='BMU') | Q(iso3__icontains='CYM') |
# #     Q(iso3__icontains='FLK') | Q(iso3__icontains='GIB') | Q(iso3__icontains='IMN') |
# #     Q(iso3__icontains='IOT') | Q(iso3__icontains='JEY') | Q(iso3__icontains='MSR') |
# #     Q(iso3__icontains='PCN') | Q(iso3__icontains='SGS') | Q(iso3__icontains='SHN') |
# #     Q(iso3__icontains='TCA') | Q(iso3__icontains='VGB')
# )
wdpa_filter = (
    Q(iso3__icontains='TUV') | Q(parent_iso3__icontains='TUV')
)

wdpa_exclude = (
    Q(iso3__icontains='USA') | Q(parent_iso3__icontains='USA') |
    Q(iso3__icontains='UMI') | Q(parent_iso3__icontains='UMI') |
    Q(iso3__icontains='VIR') | Q(parent_iso3__icontains='VIR') |
    Q(iso3__icontains='PRI') | Q(parent_iso3__icontains='PRI') |
    Q(iso3__icontains='ASM') | Q(parent_iso3__icontains='ASM') |
    Q(iso3__icontains='GUM') | Q(parent_iso3__icontains='GUM') |
    Q(iso3__icontains='MNP') | Q(parent_iso3__icontains='MNP') |
    Q(iso3__icontains='MEX') | Q(iso3__icontains='CHL') | Q(iso3__icontains='IDN') | 
    Q(iso3__icontains='CHN') | Q(iso3__icontains='JPN') | 
    Q(parent_iso3__icontains='MEX') | Q(parent_iso3__icontains='CHL') | Q(parent_iso3__icontains='IDN') |
    Q(parent_iso3__icontains='CHN') | Q(parent_iso3__icontains='JPN')
)
polys = Wdpa2019Poly.objects.exclude(wdpa_pid__in=allpid2add).exclude(wdpa_exclude).filter(wdpa_filter).filter(marine__in=('1','2')).order_by('wdpa_pid').values_list('wdpa_pid', flat=True)
points = Wdpa2019Point.objects.exclude(wdpa_pid__in=allpid2add).exclude(wdpa_exclude).filter(wdpa_filter).filter(marine__in=('1','2')).order_by('wdpa_pid').values_list('wdpa_pid', flat=True)
allpid2update = list(set(list(polys) + list(points)))

polys_nonmarine = Wdpa2019Poly.objects.exclude(wdpa_pid__in=allpid2add).exclude(wdpa_exclude).filter(wdpa_filter).filter(marine__in=('0')).order_by('wdpa_pid').values_list('wdpa_pid', flat=True)
points_nonmarine = Wdpa2019Point.objects.exclude(wdpa_pid__in=allpid2add).exclude(wdpa_exclude).filter(wdpa_filter).filter(marine__in=('0')).order_by('wdpa_pid').values_list('wdpa_pid', flat=True)
allpid2update_nonmarine = list(set(list(polys_nonmarine) + list(points_nonmarine)))

allpid2update_nonmarine_inmpatlas = mpaset.filter(wdpa_pid__in=allpid2update_nonmarine).order_by('wdpa_pid').values_list('wdpa_pid', flat=True)

allpid2update_marine012 = allpid2update + list(allpid2update_nonmarine_inmpatlas)

In [27]:
print('marine:', len(allpid2update), 'nonmarine:', len(allpid2update_nonmarine), 'all:', len(allpid2update_marine012))
print('POINTS --', 
      'marine:', Wdpa2019Point.objects.filter(wdpa_pid__in=allpid2update).count(),
      'nonmarine:', Wdpa2019Point.objects.filter(wdpa_pid__in=allpid2update_nonmarine).count()
)
print('POLYGONS --', 
      'marine:', Wdpa2019Poly.objects.filter(wdpa_pid__in=allpid2update).count(),
      'nonmarine:', Wdpa2019Poly.objects.filter(wdpa_pid__in=allpid2update_nonmarine).count(),
)
print('MPATLAS Records --', 
      'marine:', mpaset.filter(wdpa_pid__in=allpid2update).count(),
      'nonmarine:', mpaset.filter(wdpa_pid__in=allpid2update_nonmarine).count(),
)

marine: 0 nonmarine: 0 all: 0
POINTS -- marine: 0 nonmarine: 0
POLYGONS -- marine: 0 nonmarine: 0
MPATLAS Records -- marine: 0 nonmarine: 0


In [28]:
len(set(allpid2update_marine012))

0

In [29]:
# All marine=1,2 sites and only marine=0 that exist in MPAtlas
updatepoints = Wdpa2019Point.objects.filter(wdpa_pid__in=allpid2update_marine012).order_by('wdpa_pid')
merge.updateMpasFromWdpaQueryset(qs=updatepoints, poly=False, logfile=None, geologfile=updatepoint_geolog, dryrun=False)

True

In [30]:
reload(merge)
# All marine=1,2 sites and only marine=0 that exist in MPAtlas
updatepolys = Wdpa2019Poly.objects.filter(wdpa_pid__in=allpid2update_marine012).order_by('wdpa_pid')
#updatepolys = updatepolys.exclude(wdpa_pid__in=partialrunids)
merge.updateMpasFromWdpaQueryset(qs=updatepolys, poly=True, logfile=updatepoly_log, geologfile=None, dryrun=False)

True

In [31]:
mpids = mpaset.filter(wdpa_id__isnull=False, wdpa_pid__isnull=False)
mpids_count = mpids.count()
count = 0
for m in mpids:
    count += 1
    try:
        w = Wdpa2019Poly.objects.filter(wdpa_pid=m.wdpa_pid)
    except:
        try:
            w = Wdpa2019Point.objects.filter(wdpa_pid=m.wdpa_pid)
        except:
            pass
    if w.exists():
        w0 = w[0]
        mq = mpaset.filter(pk=m.pk)
        print('%s/%s mpa_id %s wdpa_pid %s updated with _wdpa values' % (count, mpids_count, m.mpa_id, w0.wdpa_pid))
        mq.update(
            no_take_wdpa=w0.no_take,
            no_take_area_wdpa=w0.no_tk_area,
        )

1/18 mpa_id 68817106 wdpa_pid 555651494 updated with _wdpa values
2/18 mpa_id 68817107 wdpa_pid 555651495 updated with _wdpa values
3/18 mpa_id 68815949 wdpa_pid 555547861_C updated with _wdpa values
4/18 mpa_id 14509 wdpa_pid 555547862_A updated with _wdpa values
5/18 mpa_id 68815950 wdpa_pid 555547862_B updated with _wdpa values
6/18 mpa_id 68815951 wdpa_pid 555547862_C updated with _wdpa values
7/18 mpa_id 14510 wdpa_pid 555547863_A updated with _wdpa values
8/18 mpa_id 68815952 wdpa_pid 555547863_B updated with _wdpa values
9/18 mpa_id 68815955 wdpa_pid 555547865_B updated with _wdpa values
10/18 mpa_id 68815953 wdpa_pid 555547864_B updated with _wdpa values
11/18 mpa_id 68815954 wdpa_pid 555547864_C updated with _wdpa values
12/18 mpa_id 68817105 wdpa_pid 555651493_B updated with _wdpa values
13/18 mpa_id 14511 wdpa_pid 555547864_A updated with _wdpa values
14/18 mpa_id 14508 wdpa_pid 555547861_A updated with _wdpa values
15/18 mpa_id 68815948 wdpa_pid 555547861_B updated with _wd

mpaset.filter(wdpa_pid__isnull=False).update(
    no_take_wdpa=F('no_take'), no_take_area_wdpa=F('no_take_area')
)

In [32]:
nt = ('All', 'Part', 'None')
ntupdate = mpaset.filter(wdpa_pid__isnull=False).filter(
    (~Q(no_take_wdpa__in=('All')) & Q(no_take_mpatlas__in=('All')))
    | (~Q(no_take_wdpa__in=('All', 'Part')) & Q(no_take_mpatlas__in=('All', 'Part')))
    | (Q(no_take_wdpa__in=('Not Applicable')) & Q(no_take_mpatlas__in=('All', 'Part', 'None')))
)
ntupdate.update(no_take=F('no_take_mpatlas'))

2

In [33]:
ntareaupdate = mpaset.filter(wdpa_pid__isnull=False).filter(
    Q(no_take_area_mpatlas__isnull=False) &
    (
        Q(no_take_area_wdpa__isnull=True)
        | Q(no_take_area_mpatlas__gt=F('no_take_area_wdpa'))
    )
)
ntareaupdate.update(no_take_area=F('no_take_area_mpatlas'))

2

In [34]:
ntchanges = ntupdate | ntareaupdate
for n in ntchanges:
    print(n.mpa_id, n.wdpa_id, n.wdpa_pid, n.name, n.designation, n.designation_eng, n.no_take_wdpa, n.no_take_mpatlas, n.no_take_area_wdpa, n.no_take_area_mpatlas)

14508 555547861 555547861_A Nukufetau Locally Managed Marine Area Locally Managed Marine Area Locally Managed Marine Area None All 0.0 11.0
14510 555547863 555547863_A Nanumea Locally Managed Marine Area Locally Managed Marine Area Locally Managed Marine Area None All 0.0 2.0


In [35]:
ntchanges = ntupdate | ntareaupdate

import csv

nt_log = '/home/mpatlas/workspace/log/notakechange_TUV_log.csv'
# nt_log = '/Users/russmo/Code/wdpa/log/notakechange_TUV_log.csv'
with open(nt_log, 'w', newline='') as csvfile:
    fieldnames = ['mpa_id', 'wdpa_id', 'wdpa_pid', 'name', 'designation', 'designation_eng', 'no_take_wdpa', 'no_take_mpatlas', 'no_take_area_wdpa', 'no_take_area_mpatlas']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for row in ntchanges.values(*fieldnames):
        writer.writerow(row)

In [74]:
for n in ntchanges:
    try:
        w2019 = Wdpa2019Poly.objects.get(wdpa_pid=n.wdpa_pid)
    except:
        w2019 = Wdpa2019Point.objects.get(wdpa_pid=n.wdpa_pid)
    try:
        w2018 = Wdpa2018Polygon.objects.filter(wdpa_pid=n.wdpa_pid)[0]
    except:
        try:
            w2018 = Wdpa2018Point.objects.filter(wdpa_pid=n.wdpa_pid)[0]
        except:
            try:
                w2018 = Wdpa2018Polygon.objects.filter(wdpaid=n.wdpa_id)[0]
            except:
                w2018 = Wdpa2018Point.objects.filter(wdpaid=n.wdpa_id)[0]
    if (n.no_take == w2018.no_take) or (n.no_take_area == w2018.no_tk_area):
        print(w2019.name, w2019.desig, n.no_take_mpatlas, w2019.no_take, w2018.no_take, n.no_take_area_mpatlas, w2019.no_tk_area, w2018.no_tk_area)

IndexError: list index out of range

In [87]:
ntchanges2 = ntupdate | ntareaupdate
ntchanges.count()

6

In [95]:
mpaset = mpas_all_nogeom.exclude(
        Q(country__icontains='USA') | Q(sovereign__icontains='USA') |
        Q(country__icontains='UMI') | Q(sovereign__icontains='UMI') |
        Q(country__icontains='VIR') | Q(sovereign__icontains='VIR') |
        Q(country__icontains='PRI') | Q(sovereign__icontains='PRI') |
        Q(country__icontains='ASM') | Q(sovereign__icontains='ASM') |
        Q(country__icontains='GUM') | Q(sovereign__icontains='GUM') |
        Q(country__icontains='MNP') | Q(sovereign__icontains='MNP')
    ).exclude(
        Q(country__icontains='MEX') | Q(country__icontains='CHL') | Q(country__icontains='IDN') | 
        Q(country__icontains='CHN') | Q(country__icontains='JPN') | 
        Q(sovereign__icontains='MEX') | Q(sovereign__icontains='CHL') | Q(sovereign__icontains='IDN') |
        Q(sovereign__icontains='CHN') | Q(sovereign__icontains='JPN')
    ).exclude(
        Q(country__icontains='FRA') | Q(sovereign__icontains='FRA') |
        Q(country__icontains='ATF') | Q(country__icontains='BLM') | Q(country__icontains='GLP') |
        Q(country__icontains='GUF') | Q(country__icontains='MAF') | Q(country__icontains='MTQ') |
        Q(country__icontains='MYT') | Q(country__icontains='NCL') | Q(country__icontains='PYF') |
        Q(country__icontains='REU') | Q(country__icontains='SHN') | Q(country__icontains='SYC')
    ).exclude(
        Q(country__icontains='AUS') | Q(sovereign__icontains='AUS') |
        Q(country__icontains='ZAF') | Q(sovereign__icontains='ZAF') |
        Q(country__icontains='CAN') | Q(sovereign__icontains='CAN')
    ).exclude(
        Q(country__icontains='GBR') | Q(sovereign__icontains='GBR') |
        Q(country__icontains='AIA') | Q(country__icontains='BMU') | Q(country__icontains='CYM') |
        Q(country__icontains='FLK') | Q(country__icontains='GIB') | Q(country__icontains='IMN') |
        Q(country__icontains='IOT') | Q(country__icontains='JEY') | Q(country__icontains='MSR') |
        Q(country__icontains='PCN') | Q(country__icontains='SGS') | Q(country__icontains='SHN') |
        Q(country__icontains='TCA') | Q(country__icontains='VGB')
    )

from mpatlas.utils import cartompa
cartompa.purgeCartoMpas()

[188,
 195,
 498,
 500,
 504,
 512,
 513,
 514,
 515,
 995,
 1429,
 1431,
 1434,
 1435,
 1436,
 1437,
 1438,
 1440,
 1445,
 1503,
 1505,
 1508,
 1509,
 1510,
 1511,
 1513,
 1851,
 1872,
 2362,
 2363,
 2364,
 2365,
 2366,
 2367,
 2369,
 2370,
 2371,
 2375,
 2378,
 2510,
 2517,
 2518,
 2519,
 2522,
 2524,
 3582,
 3819,
 3820,
 3823,
 3824,
 6871,
 6884,
 6935,
 6936,
 6938,
 6942,
 6943,
 6945,
 6946,
 6948,
 6949,
 6952,
 6955,
 6956,
 6957,
 7270,
 7283,
 7299,
 9280,
 9397,
 9398,
 9399,
 9400,
 9401,
 9402,
 9403,
 9405,
 9406,
 9407,
 9408,
 9409,
 9410,
 9411,
 9412,
 9413,
 9414,
 9415,
 9416,
 9417,
 9418,
 9419,
 9420,
 9421,
 9422,
 15058,
 15083,
 7700994,
 7700999,
 7701851,
 7702510,
 7702516,
 7702520,
 7702521,
 7703821,
 7703822,
 7703825,
 7703826,
 7705349,
 7709259]

In [94]:
cartompa.updateAllMpas(mpas=mpaset)

Processing 10 of 1987 mpa records at a time
Records [0 - 9]
Records [10 - 19]
Records [20 - 29]
Records [30 - 39]
ERROR processing mpa 68807503:  'NoneType' object has no attribute 'hexewkb'
Skipping Mpa 68807503
Records [40 - 49]
ERROR processing mpa 68807520:  'NoneType' object has no attribute 'hexewkb'
Skipping Mpa 68807520
ERROR processing mpa 68807504:  'NoneType' object has no attribute 'hexewkb'
Skipping Mpa 68807504
Records [50 - 59]
Records [60 - 69]
ERROR processing mpa 68807500:  'NoneType' object has no attribute 'hexewkb'
Skipping Mpa 68807500
ERROR processing mpa 68807495:  'NoneType' object has no attribute 'hexewkb'
Skipping Mpa 68807495
ERROR processing mpa 60009586:  'NoneType' object has no attribute 'hexewkb'
Skipping Mpa 60009586
ERROR processing mpa 60009547:  'NoneType' object has no attribute 'hexewkb'
Skipping Mpa 60009547
ERROR processing mpa 60009546:  'NoneType' object has no attribute 'hexewkb'
Skipping Mpa 60009546
ERROR processing mpa 60009495:  'NoneTyp

KeyboardInterrupt: 